In [6]:
from time import sleep
import Pyro5.api 
import matplotlib.pyplot as plt
from TimeTagger import *

In [7]:
# Name of the file we want to read. 
# Remember, "Measurement.ttbin" files read all the files in sequence 'Measurement'
file = "Measurement.ttbin"

In [8]:
fr = FileReader(file)
print("All channels available: {}".format(fr.getChannelList()))

print("Has data? ->", fr.hasData())
# Returns: True if more data is available for reading, False if all data has been read from all the files specified in the class constructor.

All channels available: (1, 2)
Has data? -> True


In [9]:
# Now we are going to print all the configuration of the TT-Ultra at the moment of the measurement
config = fr.getConfiguration()

for key, value in config.items():
    print(f"{key}: {value}")

channel number scheme: 2
configure version: 1
current time: 2024-09-19 11:57:14 +0200
devices: [{'FPGA ID': 11493476189726804, 'PCB UID': '00fdf287', 'PCB version': '1.8 (1)', 'minimum software version': '2.10.6', 'model': 'Time Tagger Ultra', 'serial': '2410001ABQ'}]
firmware version: TT-Ultra, FW6, TS 2024-02-09 12:21:10, OK 1.46
fpga link: {'enabled': False}
hardware buffer size: 67108864
inputs: [{'average rising falling': [False, False], 'channel': [1, -1], 'conditional filter filtered': [False, False], 'conditional filter triggers': [False, False], 'deadtime': [2000, 2000], 'delay hardware': [0, 0], 'delay software': [0, 0], 'event divider': [1, 1], 'hardware delay compensation': [333, 333], 'high priority channel': [False, False], 'input hysteresis': 0, 'input impedance high': False, 'input mux': 0, 'normalization': [True, True], 'resolution': 'Standard', 'resolution rms': 42.0, 'trigger level': 0.5}, {'average rising falling': [False, False], 'channel': [2, -2], 'conditional fi

### Some useful code for obtaining the timestamps on each channel

In [10]:
data = fr.getData(4)
# Reads the next n_events and returns the buffer object with the specified number of timetags.
print(fr.hasData())
# Returns: True if more data is available for reading, False if all data has been read from all the files specified in the class constructor.

print(data.getTimestamps())
# Returns: Event timestamps in picoseconds for all chosen channels.
print(data.getChannels())
# Returns: Channel number for each detected event.

print(data.tGetData)
# Returns: The data-stream time position when the TimeTagStream or FileWriter started data acquisition.
print(data.tStart)


data = fr.getData(4)
print(fr.hasData())
print(data.getTimestamps())

True
[123491014370 123502096876 123515543010 123516205922]
[1 1 2 2]
123516486063
123486208000
True
[123516486063 123520281890 123522707650 123524903322]
